In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd

In [2]:
raw_info = pd.read_csv('../data/commuting_allinfo.csv',dtype=object,encoding='cp1252')

In [3]:
raw_info.head()

,State FIPS Code,County FIPS Code,State Name,County Name,Metropolitan Statistical Area FIPS Code,Metropolitan Statistical Area of County,State FIPS Code.1,County FIPS Code.1,State Name.1,County Name.1,Metropolitan Statistical Area FIPS Code.1,Metropolitan Statistical Area of County,Workers in Commuting Flow,Margin of Error
0,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,001,Alabama,Autauga County,33860,"Montgomery, AL","8,635",597
1,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,007,Alabama,Bibb County,13820,"Birmingham-Hoover, AL",16,23
2,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,013,Alabama,Butler County,NaN,NaN,4,7
3,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,021,Alabama,Chilton County,13820,"Birmingham-Hoover, AL",597,251
4,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,043,Alabama,Cullman County,NaN,NaN,27,42


In [4]:
raw_info.columns

Index(['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'Metropolitan Statistical Area FIPS Code',
       'Metropolitan Statistical Area  of County', 'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Metropolitan Statistical Area FIPS Code.1',
       ' Metropolitan Statistical Area of County', 'Workers in Commuting Flow',
       'Margin of Error'],
      dtype='object')

In [3]:
columns = ['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'Metropolitan Statistical Area FIPS Code',
       'Metropolitan Statistical Area  of County', 'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Metropolitan Statistical Area FIPS Code.1',
       ' Metropolitan Statistical Area of County', 'Workers in Commuting Flow',
       'Margin of Error']

In [84]:
for name in columns[:11]:
    print('name:', name)
    print('na: ', sum(raw_info[name].isna()))
    print('length of unique: ', len(set(raw_info[name])))
    print("_______________")

name: State FIPS Code
na:  1
length of unique:  55
_______________
name: County FIPS Code
na:  3
length of unique:  326
_______________
name: State Name
na:  3
length of unique:  53
_______________
name: County Name
na:  3
length of unique:  1956
_______________
name: Metropolitan Statistical Area FIPS Code
na:  57298
length of unique:  394
_______________
name: Metropolitan Statistical Area  of County
na:  57298
length of unique:  394
_______________
name: State FIPS Code.1
na:  1200
length of unique:  55
_______________
name: County FIPS Code.1
na:  1790
length of unique:  326
_______________
name: State Name.1
na:  3
length of unique:  56
_______________
name: County Name.1
na:  1790
length of unique:  1956
_______________
name: Metropolitan Statistical Area FIPS Code.1
na:  49509
length of unique:  394
_______________


In [4]:
## delete all the nan row
county_travel =  raw_info[['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
        'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Workers in Commuting Flow',
       'Margin of Error']]

In [5]:
county_travel_cleaned = county_travel.dropna()

In [8]:
# data_types_dict = {'State FIPS Code':np.int32, 'County FIPS Code':np.int32, 
#         'State FIPS Code.1':np.int32,
#        'County FIPS Code.1':np.int32,
#        'Workers in Commuting Flow':np.float64,
#        'Margin of Error':np.float64}

In [6]:
df = pd.DataFrame()
for name in ['State FIPS Code', 'County FIPS Code', 
        'State FIPS Code.1',
       'County FIPS Code.1', 
       'Workers in Commuting Flow',
       'Margin of Error']:
    df[name] =  county_travel_cleaned[name].str.replace(',', '').astype(float) 

In [7]:
df['longfips'] = df['State FIPS Code']*1000+df['County FIPS Code']
df['longfips1'] = df['State FIPS Code.1']*1000+df['County FIPS Code.1']

In [8]:
df.head()

,State FIPS Code,County FIPS Code,State FIPS Code.1,County FIPS Code.1,Workers in Commuting Flow,Margin of Error,longfips,longfips1
0,1.0,1.0,1.0,1.0,8635.0,597.0,1001.0,1001.0
1,1.0,1.0,1.0,7.0,16.0,23.0,1001.0,1007.0
2,1.0,1.0,1.0,13.0,4.0,7.0,1001.0,1013.0
3,1.0,1.0,1.0,21.0,597.0,251.0,1001.0,1021.0
4,1.0,1.0,1.0,43.0,27.0,42.0,1001.0,1043.0


In [9]:
cn_all = list(df.longfips)+list(df.longfips1)

In [10]:
len(set(cn_all))
### it is larger the fips_number within US

3221

In [11]:
fips = loadmat('../data/countyfips_num.mat') ### dictionary

In [12]:
fips_num = fips['countyfips_num'].ravel() ### is the sorted, the index is corresponded to the position of fips_number
is_sorted = lambda a: np.all(a[:-1] <= a[1:])
print(is_sorted(fips_num))

True


In [13]:
no_fips = len(fips_num) ### the number of fips is 3142

In [16]:
set(fips_num) - set(cn_all)
## new in the fips code 2158 (Kusilvak Census Area) previously as 
#  02270 (Alaska      Wade Hampton Census Area)
#^__________________________________________^#
## 46102 (Oglala Lakota County) previously as 
# 46113 (South Dakota      Shannon County)
#^__________________________________________^#
## 51515 Virginia   Bedford city is 
# Virginia, 2013: The independent city of Bedford (FIPS 51515) merges into
# Bedford County (FIPS 51019).

{2158, 46102}

In [14]:
### change the code
df.loc[df["longfips"] == 46113, "longfips"] = 46102
df.loc[df["longfips1"] == 46113, "longfips1"] = 46102
df.loc[df["longfips"] == 2270, "longfips"] = 2158
df.loc[df["longfips1"] == 2270, "longfips1"] = 2158

In [19]:
cn_all = list(df.longfips)+list(df.longfips1) ## calculate again
set(fips_num) - set(cn_all) ## there is no new codes

set()

In [20]:
np.sum(df.isna()) ### there is no nan

State FIPS Code              0
County FIPS Code             0
State FIPS Code.1            0
County FIPS Code.1           0
Workers in Commuting Flow    0
Margin of Error              0
longfips                     0
longfips1                    0
dtype: int64

In [15]:
## merge the code into the larger county
df.loc[df["longfips"] == 51515, "longfips"] = 51019
df.loc[df["longfips1"] == 51515, "longfips1"] = 51019
### there will be a duplicate ones, and merge them again

In [16]:
temp = df[(df['longfips'] == 51019)|(df['longfips1'] == 51019)][['longfips',	'longfips1']]

In [33]:
temp ### there is no need to merge it now, when create the M matrix it can count dumplicate entries twice.

,longfips,longfips1
10560,8013.0,51019.0
15299,12057.0,51019.0
17314,12103.0,51019.0
19062,13047.0,51019.0
81443,37067.0,51019.0
...,...,...
126062,51680.0,51019.0
126089,51680.0,51019.0
126158,51690.0,51019.0
126609,51770.0,51019.0


In [17]:
fips_id = [int(i) for i in range(len(fips_num))]
fips_dict = dict(zip(fips_num,fips_id))

In [18]:
df['fips_id']=df['longfips']
df.replace({'fips_id':fips_dict},inplace=True)
# df['fips1_id']=df['longfips1'].map(fips_dict) ## map is one to one
df['fips1_id']=df['longfips1']
df.replace({'fips1_id':fips_dict},inplace=True)

In [142]:
# df_temp1 = pd.merge(df,df_fips,left_on='longfips',right_on='fips',how='outer')
# df_temp1.drop(['fips'],axis=1,inplace=True)
# df_temp2 = pd.merge(df_temp1,df_fips,left_on='longfips1',right_on='fips',how='outer')
## the order has been changed 
## prefer not using this one

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [27]:
### loop over the commuting infomation
### the data in puerto rico does not need to take into consideration
M = np.zeros((no_fips,no_fips))
for com_i in range(len(df)):
#     print(com_i)
    residence = df['longfips'].iloc[com_i]
    work = df['longfips1'].iloc[com_i] ### it is not very necessary
    com = df['Workers in Commuting Flow'].iloc[com_i]

    
    if (residence in fips_num) and (work in fips_num): #(previously selected)
    
        residence_id = np.where(fips_num == residence)[0][0]
        work_id = np.where(fips_num == work)[0][0]
        
        if M[work_id,residence_id] == 0:
            M[work_id,residence_id] =  com
        else:
            print(com_i)
            print('work:', work, work_id, 'residence:', residence, residence_id)
            M[work_id,residence_id] = M[work_id,residence_id] + com
            print ("*^_____^*")

119918
work: 51019.0 2829 residence: 51009.0 2824
*^_____^*
119951
work: 51019.0 2829 residence: 51011.0 2825
*^_____^*
120145
work: 51019.0 2829 residence: 51019.0 2829
*^_____^*
120199
work: 51019.0 2829 residence: 51023.0 2831
*^_____^*
120325
work: 51019.0 2829 residence: 51031.0 2835
*^_____^*
121173
work: 51019.0 2829 residence: 51067.0 2852
*^_____^*
122645
work: 51019.0 2829 residence: 51143.0 2888
*^_____^*
123021
work: 51019.0 2829 residence: 51161.0 2896
*^_____^*
123059
work: 51019.0 2829 residence: 51163.0 2897
*^_____^*
123887
work: 51009.0 2824 residence: 51019.0 2829
*^_____^*
123888
work: 51019.0 2829 residence: 51019.0 2829
*^_____^*
123889
work: 51023.0 2831 residence: 51019.0 2829
*^_____^*
123890
work: 51031.0 2835 residence: 51019.0 2829
*^_____^*
123891
work: 51067.0 2852 residence: 51019.0 2829
*^_____^*
123892
work: 51121.0 2879 residence: 51019.0 2829
*^_____^*
123893
work: 51143.0 2888 residence: 51019.0 2829
*^_____^*
123894
work: 51161.0 2896 residence: 510

In [33]:
df[(df['longfips'] == 51009)&(df['longfips1'] == 51019)] ## a simple check if the 51515 is merged into the 51019

,State FIPS Code,County FIPS Code,State FIPS Code.1,County FIPS Code.1,Workers in Commuting Flow,Margin of Error,longfips,longfips1,fips_id,fips1_id
121499,51.0,9.0,51.0,19.0,709.0,170.0,51009.0,51019.0,2824.0,2829.0
121514,51.0,9.0,51.0,515.0,83.0,71.0,51009.0,51019.0,2824.0,2829.0


In [35]:
## compare with the matlab verison:
M_m = loadmat('../data/M.mat') ### dictionary
list(M_m.keys())
M_m = M_m['M']

In [39]:
np.sum(M_m != M) ###only some of the entries are not equal, due to the fips merge/change problems. 

57

In [40]:
np.savetxt('M_all_asymmetric.csv',M)